<a href="https://colab.research.google.com/github/lauraluebbert/test_gget_alphafold/blob/main/test_gget_alphafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [gget alphafold](https://github.com/pachterlab/gget) test

___

Install and import gget:

In [ ]:
!pip install -U -q gget
import gget

Install OpenMM:  

In [ ]:
# For pretty plots
%config InlineBackend.figure_format='retina'

import os
import sys

python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
miniforge_dir = os.path.expanduser("~/miniforge")
mamba_bin = f"{miniforge_dir}/bin/mamba"
miniforge_python = f"{miniforge_dir}/bin/python"
site_packages = f"{miniforge_dir}/lib/python{python_version}/site-packages"

# Install Miniforge
os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh")
os.system(f"bash Miniforge3-Linux-x86_64.sh -bfp {miniforge_dir}")

# Install dependencies via mamba into Miniforge
os.system(
    f"{mamba_bin} install -y -c conda-forge -c bioconda "
    f"openmm=8.2.0 python='{python_version}' pip numpy=1.24 scipy jax=0.4.26 jaxlib=0.4.26"
)

# Install gget into Miniforge's Python
os.system(f"{miniforge_python} -m pip install -U gget")

# Append Miniforge site-packages to current Python path
sys.path.insert(0, site_packages)

Install third-part dependencies and download AlphaFold model parameters using `gget setup` (this might take a few minutes):

In [ ]:
gget.setup("alphafold", force=True)


Predict the 3D structure of CASP14 target [T1024](https://predictioncenter.org/casp14/target.cgi?id=8&view=all):

In [ ]:
gget.alphafold(
    "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH",
    relax=True,
    out=None
    )